In [2]:
## IMPORTS

import os
import sys
import time
import math
from datetime import datetime
from io import StringIO


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from numpy.polynomial.polynomial import Polynomial
import scipy.integrate as integrate
import scipy.optimize as sco
from scipy.optimize import minimize
from sklearn.metrics import mean_squared_error
import openpyxl
import networkx as nx
import sklearn.metrics
from IPython.core.display import HTML
import plotly.io as pio
from openpyxl.utils.dataframe import dataframe_to_rows
import openpyxl as opxl
import matplotlib.ticker as mtick
from matplotlib.cm import get_cmap
import time


sys.path.append(r'D:\seagu_OneDrive\Documents\GitHub\parallel-flow-distribution-pressure-loss\ansys')
sys.path.append(r'D:\seagu_OneDrive\Documents\GitHub\PVT-thermal-performance-model')
import jou_gen as jg
import ansys_py_bridge as apb
import ansys.fluent.core as pyfluent
import model as ty
import proc as pr
import plot_functions_here as pfun
import heat_transfer as bht
import fluids as fds
import ht
import general as gen

# Visualisation des résultats

In [3]:
hypotheses_path = r'D:\ANSYS Fluent Projects\pvt_slice_outdoor\Fluent_GMI\tests' + '\\' + 'Model_hypotheses.xlsx'
hyp = pr.create_dict_from_excel(hypotheses_path,'Main')

nb_mesh = 3
nb_cases = 5
limit_big_it = 6
nb_hx = 5

In [4]:
## Récupération des données sauvegardées

folder_path = r'D:\ANSYS Fluent Projects\pvt_slice_outdoor\Fluent_GMI\tests'

ht_tot_mesh_case_list = []
ht_rad_mesh_case_list = []
ht_conv_mesh_case_list = []
CFD_mesh_case_list = []
df_one_mesh_case_list = []
slices_df_mesh_case_list = []
PyFluent_mesh_case_list = []

for mesh in range(nb_mesh) :

    folder_mesh = f'Simu_M{mesh+1}'
    folder_path_mesh = os.path.join(folder_path, folder_mesh)

    ht_tot_case_list = []
    ht_rad_case_list = []
    ht_conv_case_list = []
    CFD_case_list = []
    df_one_case_list = []
    slices_df_case_list = []
    PyFluent_case_list = []

    for case in range(nb_cases) :
    # case = 13
    # if True :
        ## Récupération des résultats

        ht_tot_list = []
        ht_rad_list = []
        ht_conv_list = []
        CFD_list = []
        df_one_list = []
        slices_df_list = []
        PyFluent_list = []

        # Récupération des données pour chaque cas
        folder_name = f'cas_{case+1}'
        folder_path_case = os.path.join(folder_path_mesh, folder_name)

        # Définir le chemin du dossier pour les fichiers de résultats
        hyp['CFD_ht_path'] = os.path.join(folder_path_case, 'test')

        # for iteration in range(1, limit_big_it+1) :
        for iteration in range(limit_big_it) :
            file_path_result_CFD = hyp['CFD_ht_path']+f'_{iteration}.csv' 
            file_path_df_one = hyp['CFD_ht_path']+'_df_one' + f'_{iteration}.csv'
            file_path_slices_df = hyp['CFD_ht_path']+'_slices_df' + f'_{iteration}.csv'
            file_path_Inputs_PyFluent = hyp['CFD_ht_path']+'_PyFluent' + f'_{iteration}.csv'

            df_CFD = pd.read_csv(file_path_result_CFD, sep=';')
            df_CFD['iteration'] = iteration
            df_CFD.index = [f'part{i}' for i in range(1, nb_hx+3)]
            CFD_list.append(df_CFD)

            df_one = pd.read_csv(file_path_df_one, sep=';')
            df_one['iteration'] = iteration
            df_one.index = [f'part{i}' for i in range(1, nb_hx+3)]
            df_one_list.append(df_one)

            slices_df = pd.read_csv(file_path_slices_df, sep=';')
            slices_df['iteration'] = iteration
            slices_df.index = [f'part{i}' for i in range(1, nb_hx+3)]
            slices_df_list.append(slices_df)

            df_PyFluent = pd.read_csv(file_path_Inputs_PyFluent, sep=';')
            df_PyFluent['iteration'] = iteration
            PyFluent_list.append(df_PyFluent)

            ht_tot = pd.read_csv(os.path.join(folder_path_case,f'ht_tot_report_{iteration}.csv'), sep=',')
            ht_tot['iteration'] = iteration
            ht_tot_list.append(ht_tot)

            ht_rad = pd.read_csv(os.path.join(folder_path_case,f'ht_rad_report_{iteration}.csv'), sep=',')
            ht_rad['iteration'] = iteration
            ht_rad_list.append(ht_rad)
            
            ht_conv = pd.read_csv(os.path.join(folder_path_case,f'ht_conv_report_{iteration}.csv'), sep=';')
            ht_conv['iteration'] = iteration
            ht_conv_list.append(ht_conv)

        ht_tot_case_list.append(ht_tot_list)
        ht_rad_case_list.append(ht_rad_list)
        ht_conv_case_list.append(ht_conv_list)
        CFD_case_list.append(CFD_list)
        df_one_case_list.append(df_one_list)
        slices_df_case_list.append(slices_df_list)
        PyFluent_case_list.append(PyFluent_list)

    ht_tot_mesh_case_list.append(ht_tot_case_list)
    ht_rad_mesh_case_list.append(ht_rad_case_list)
    ht_conv_mesh_case_list.append(ht_conv_case_list)
    CFD_mesh_case_list.append(CFD_case_list)
    df_one_mesh_case_list.append(df_one_case_list)
    slices_df_mesh_case_list.append(slices_df_case_list)
    PyFluent_mesh_case_list.append(PyFluent_case_list)

In [5]:
## Calcul des transferts à partir des reports // CFD
# Définir les parties pour chaque transfert
part1 = ['manifold_yu']
part2 = ['hx_bend_yu_air', 'hx_bend_yu_pv']
part3 = ['hx_flat_yu_air']
part4 = ['hx_bend_mid_air', 'hx_bend_mid_pv']
part5 = ['hx_flat_yd_air']
part6 = ['hx_bend_yd_air', 'hx_bend_yd_pv']
part7 = ['manifold_yd']

part1_top = []
part2_top = []
part3_top = ['hx_flat_yu_pv-pv_backsheet-cd-cd1-pv-corps']
part4_top = []
part5_top = ['hx_flat_yd_pv-pv_backsheet-cd-cd1-pv-corps']
part6_top = []
part7_top = []

# Fonction pour calculer les Qdot pour chaque partie et chaque itération
def calculate_Qdot(mesh, case, iteration):
    ht_tot = ht_tot_mesh_case_list[mesh][case][iteration]
    ht_rad = ht_rad_mesh_case_list[mesh][case][iteration]
    ht_conv = ht_conv_mesh_case_list[mesh][case][iteration]
    
    Qdot_tube = []
    Qdot_top = []
    Qdot_abs = []
    Qdot_top_rad = []
    
    for i in range(1, nb_hx+3):
        if i == 3 or i == 5:
            Qdot_tube.append(4.75*ht_tot[ht_tot['Component'].isin(eval(f'part{i}'))]['ht'].sum())
            Qdot_top.append(4.75*ht_conv[ht_conv['Component'].isin(eval(f'part{i}_top'))]['conv_ht'].sum())
            Qdot_abs.append(1e-6)
            Qdot_top_rad.append(4.75*ht_rad[ht_rad['Component'].isin(eval(f'part{i}_top'))]['rad_ht'].sum())
        else:
            Qdot_tube.append(4.75*ht_conv[ht_conv['Component'].isin(eval(f'part{i}'))]['conv_ht'].sum())
            Qdot_top.append(1e-6)
            Qdot_abs.append(1e-6)  # Dans le modèle, on a fait en sorte de ne pas avoir besoin de cette valeur
            Qdot_top_rad.append(1e-6)
    
    return Qdot_tube, Qdot_top, Qdot_abs, Qdot_top_rad

In [90]:
## Ratio de transfert convectif et radiatif // CFD

part1 = ['manifold_yu']
part2 = ['hx_bend_yu_air', 'hx_bend_yu_pv']
part3 = ['hx_flat_yu_air']
part4 = ['hx_bend_mid_air', 'hx_bend_mid_pv']
part5 = ['hx_flat_yd_air']
part6 = ['hx_bend_yd_air', 'hx_bend_yd_pv']
part7 = ['manifold_yd']

part1_top = []
part2_top = []
part3_top = ['hx_flat_yu_pv-pv_backsheet-cd-cd1-pv-corps']
part4_top = []
part5_top = ['hx_flat_yd_pv-pv_backsheet-cd-cd1-pv-corps']
part6_top = []
part7_top = []

component_list = [part1, part2, part3, part4, part5, part6, part7]
component_list_top = [part1_top, part2_top, part3_top, part4_top, part5_top, part6_top, part7_top]

def ratio_list(component_list):
    ratio_mesh_case_it_part_list = []
    for mesh in range(nb_mesh):
        ratio_case_it_list = []
        for case in range(nb_cases):
            ratio_case_it_part_list = []
            for iteration in range(limit_big_it):
                ratio_part_list = []
                for part in component_list:
                    if part == [] :
                        ratio_conv = 0
                        ratio_rad = 0
                    else :
                        conv = ht_conv_mesh_case_list[mesh][case][iteration][ht_conv_mesh_case_list[mesh][case][iteration]['Component'].isin(part)]['conv_ht'].sum()
                        tot = ht_tot_mesh_case_list[mesh][case][iteration][ht_tot_mesh_case_list[mesh][case][iteration]['Component'].isin(part)]['ht'].sum()
                        rad = ht_rad_mesh_case_list[mesh][case][iteration][ht_rad_mesh_case_list[mesh][case][iteration]['Component'].isin(part)]['rad_ht'].sum()
    
                        ratio_conv = conv / tot
                        ratio_rad = rad / tot
                    
                    ratio_part_list.append([ratio_conv, ratio_rad])
                
                ratio_case_it_part_list.append(ratio_part_list)
            ratio_case_it_list.append(ratio_case_it_part_list)
        ratio_mesh_case_it_part_list.append(ratio_case_it_list)
    
    return ratio_mesh_case_it_part_list

In [91]:
ratio_tube_list = ratio_list(component_list)
ratio_top_list = ratio_list(component_list_top)

C:\Users\seagu\AppData\Local\Temp\ipykernel_24268\722068977.py:39: RuntimeWarning:

divide by zero encountered in scalar divide

C:\Users\seagu\AppData\Local\Temp\ipykernel_24268\722068977.py:40: RuntimeWarning:

divide by zero encountered in scalar divide

C:\Users\seagu\AppData\Local\Temp\ipykernel_24268\722068977.py:39: RuntimeWarning:

invalid value encountered in scalar divide

C:\Users\seagu\AppData\Local\Temp\ipykernel_24268\722068977.py:40: RuntimeWarning:

invalid value encountered in scalar divide



1D

In [ ]:
## Afficher des résultats de Qdot_tube_back // 1D chaque simulation

# Définition des couleurs pour chaque partie
part_colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2']

# Initialisation des listes de figures pour chaque simulation
fig_tube_lists = [[] for _ in range(nb_mesh)]

# Boucle sur chaque simulation (mesh)
for mesh in range(nb_mesh):
    for case in range(nb_cases):
        # Créer une figure pour Qdot_tube_back_conv
        fig_tube = go.Figure()

        for part in range(1, 8):  # Supposons que vous avez 7 parties numérotées de 1 à 7
            tube_x = []
            tube_y = []

            for iteration in range(limit_big_it):
                # Récupérer les valeurs pour Qdot_tube_back_conv pour cette partie, cette simulation et cette itération
                tube_value = df_one_mesh_case_list[mesh][case][iteration]['Qdot_tube_back'].loc[f'part{part}']

                tube_x.append(iteration + 1)
                tube_y.append(tube_value)

            # Ajouter les valeurs à la figure Qdot_tube_back_conv
            fig_tube.add_trace(go.Scatter(x=tube_x, y=tube_y,
                                          mode='lines+markers',
                                          name=f'Partie {part}',
                                          line=dict(width=2, color=part_colors[part-1])))

        # Mise en forme du titre et des axes pour Qdot_tube_back_conv
        fig_tube.update_layout(title=f'Qdot_tube_back - Cas {case+1} - Simu M{mesh+1}',
                               xaxis_title='Itérations',
                               yaxis_title='Qdot_tube_back [W]',
                               plot_bgcolor='rgba(0,0,0,0)')  # Fond transparent pour superposer les graphiques

        # Ajouter la figure Qdot_tube_back_conv à la liste correspondante à la simulation actuelle
        fig_tube_lists[mesh].append(fig_tube)

# Afficher toutes les figures pour Qdot_tube_back_conv par simulation
for mesh in range(nb_mesh):
    for fig_tube in fig_tube_lists[mesh]:
        fig_tube.show()


In [ ]:
## Affichage des résultats de Qdot_top_conv pour chaque cas // 1D pour chaque simulation

# Créer une liste de figures pour chaque cas
fig_top_list = []

# Couleurs pour chaque partie (7 couleurs différentes)
part_colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2']

for mesh in range(nb_mesh):
    for case in range(nb_cases):

        # Créer une figure pour Qdot_top_conv
        fig_top = go.Figure()
        
        for part in range(1, nb_hx+3):  # Supposons que vous avez 7 parties numérotées de 1 à 7
            top_x = []
            top_y = []
            
            for iteration in range(limit_big_it):
                # Récupérer les valeurs pour Qdot_top_conv pour cette partie et cette itération
                top_value = df_one_mesh_case_list[mesh][case][iteration]['Qdot_top_conv'].loc[f'part{part}']
                
                tube_x.append(iteration + 1)
                tube_y.append(tube_value)
                top_x.append(iteration + 1)
                top_y.append(top_value)

            
            # Ajouter les valeurs à la figure Qdot_top_conv
            fig_top.add_trace(go.Scatter(x=top_x, y=top_y,
                                        mode='lines+markers',
                                        name=f'Partie {part}',
                                        line=dict(width=2, color=part_colors[part-1])))
        
        # Mise en forme du titre et des axes pour Qdot_top_conv
        fig_top.update_layout(title=f'Qdot_top_conv - Cas {case+1} - Simu M{mesh+1}',
                            xaxis_title='Itérations',
                            yaxis_title='Qdot_top_conv')

        
        # Ajouter la figure Qdot_top_conv à la liste
        fig_top_list.append(fig_top)

# Afficher toutes les figures pour Qdot_top_conv par cas
for fig_top in fig_top_list:
    fig_top.show()


In [ ]:
## Affichage des résultats de Qdot_top_rad pour chaque cas // 1D pour chaque simulation

# Créer une liste de figures pour chaque cas
fig_top_list = []

# Couleurs pour chaque partie (7 couleurs différentes)
part_colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2']

for mesh in range(nb_mesh):
    for case in range(nb_cases):

        # Créer une figure pour Qdot_top_conv
        fig_top = go.Figure()
        
        for part in range(1, nb_hx+3):  # Supposons que vous avez 7 parties numérotées de 1 à 7
            top_x = []
            top_y = []
            
            for iteration in range(limit_big_it):
                # Récupérer les valeurs pour Qdot_top_conv pour cette partie et cette itération
                top_value = df_one_mesh_case_list[mesh][case][iteration]['Qdot_top_rad'].loc[f'part{part}']
    
                top_x.append(iteration + 1)
                top_y.append(top_value)

            
            # Ajouter les valeurs à la figure Qdot_top_conv
            fig_top.add_trace(go.Scatter(x=top_x, y=top_y,
                                        mode='lines+markers',
                                        name=f'Partie {part}',
                                        line=dict(width=2, color=part_colors[part-1])))
        
        # Mise en forme du titre et des axes pour Qdot_top_conv
        fig_top.update_layout(title=f'Qdot_top_rad - Cas {case+1} - Simu M{mesh+1}',
                            xaxis_title='Itérations',
                            yaxis_title='Qdot_top_conv')

        
        # Ajouter la figure Qdot_top_conv à la liste
        fig_top_list.append(fig_top)

# Afficher toutes les figures pour Qdot_top_conv par cas
for fig_top in fig_top_list:
    fig_top.show()


In [ ]:
## Affichage des résultats de Qdot_tube_fluid pour chaque cas // 1D pour chaque simulation

# Créer une liste de figures pour chaque cas
fig_top_list = []

# Couleurs pour chaque partie (7 couleurs différentes)
part_colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2']

for mesh in range(nb_mesh):
    for case in range(nb_cases):

        # Créer une figure pour Qdot_tube_fluid
        fig_top = go.Figure()
        
        for part in range(1, nb_hx+3):  # Supposons que vous avez 7 parties numérotées de 1 à 7
            tube_x = []
            tube_y = []
            
            for iteration in range(limit_big_it):
                # Récupérer les valeurs pour Qdot_tube_fluid pour cette partie et cette itération
                tube_value = df_one_mesh_case_list[mesh][case][iteration]['Qdot_tube_fluid'].loc[f'part{part}']
    
                tube_x.append(iteration + 1)
                tube_y.append(tube_value)

            
            # Ajouter les valeurs à la figure Qdot_tube_fluid
            fig_top.add_trace(go.Scatter(x=tube_x, y=tube_y,
                                        mode='lines+markers',
                                        name=f'Partie {part}',
                                        line=dict(width=2, color=part_colors[part-1])))
        
        # Mise en forme du titre et des axes pour Qdot_tube_fluid
        fig_top.update_layout(title=f'Qdot_tube_fluid - Cas {case+1} - Simu M{mesh+1}',
                            xaxis_title='Itérations',
                            yaxis_title='Qdot_tube_fluid')

        
        # Ajouter la figure Qdot_tube_fluid à la liste
        fig_top_list.append(fig_top)

# Afficher toutes les figures pour Qdot_tube_fluid par cas
for fig_top in fig_top_list:
    fig_top.show()


In [ ]:
## Evolution des températures d'entrée et de sortie par itération // 1D pour chaque simulation

for mesh in range(nb_mesh):
    for case in range(nb_cases):
        data = []
        for iteration in range(limit_big_it):
            # Liste temporaire pour stocker les données de chaque partie et itération
            temp_data = []
            
            for part in range(1, nb_hx+3): 
                T_in_value = df_one_mesh_case_list[mesh][case][iteration]['T_fluid_in'].loc[f'part{part}']
                T_out_value = df_one_mesh_case_list[mesh][case][iteration]['T_fluid_out'].loc[f'part{part}']
                temp_data.append(T_in_value)
                temp_data.append(T_out_value)
            
            # Ajouter les données au tableau principal
            data.append(temp_data)

        # Noms des colonnes pour le tableau
        columns = []
        for part in range(1, nb_hx+3):
            columns.append(f'T_fluid_in_{part}')
            columns.append(f'T_fluid_out_{part}')

        # Création du DataFrame
        df_temperature = pd.DataFrame(data, columns=columns)
        df_temperature.index = [i for i in range(1, limit_big_it+1)]
        df_temperature = df_temperature.rename_axis('Itération')

        # Affichage du tableau
        print(f"Cas n°{case+1} - Simu_M{mesh+1}:")
        pr.disp_html(df_temperature)

CFD

In [ ]:
## Affichage Qdot_top_rad // CFD
for mesh in range(nb_mesh):
    # Boucle sur chaque cas
    for case in range(nb_cases):
        # Listes pour stocker les valeurs de chaque transfert pour chaque itération
        Qdot_top_rad_values = []

        # Boucle sur chaque itération pour calculer les valeurs
        for iteration in range(limit_big_it):
            Qdot_tube, Qdot_top, Qdot_abs, Qdot_top_rad = calculate_Qdot(mesh, case, iteration)
            Qdot_top_rad_values.append(Qdot_top_rad)
        
        # Création des figures pour chaque type de transfert
        fig_top_rad = go.Figure()

        # Ajout des traces à chaque figure
        for part in range(1, nb_hx+3):
            fig_top_rad.add_trace(go.Scatter(x=list(range(1, limit_big_it + 1)), y=[qr[part-1] for qr in Qdot_top_rad_values],
                                            mode='lines+markers',
                                            name=f'Part {part}',
                                            line=dict(width=2)))

        # Mise en forme du titre et des axes pour chaque figure
        fig_top_rad.update_layout(title=f'Qdot_top_rad - Cas {case+1} - Simu M{mesh+1}',
                                xaxis_title='Itération',
                                yaxis_title='Qdot_top_rad')

        # Affichage des figures
        fig_top_rad.show()


In [ ]:
## Affichage Qdot_top_conv // CFD

for mesh in range(nb_mesh):
    # Boucle sur chaque cas
    for case in range(nb_cases):
        # Listes pour stocker les valeurs de chaque transfert pour chaque itération
        Qdot_tube_values = []

        # Boucle sur chaque itération pour calculer les valeurs
        for iteration in range(limit_big_it):
            Qdot_tube, Qdot_top, Qdot_abs, Qdot_top_rad = calculate_Qdot(mesh, case, iteration)
            Qdot_tube_values.append(Qdot_top)

        
        # Création des figures pour chaque type de transfert
        fig_top = go.Figure()

        # Ajout des traces à chaque figure
        for part in range(1, nb_hx+3):
            fig_top.add_trace(go.Scatter(x=list(range(1, limit_big_it + 1)), y=[qt[part-1] for qt in Qdot_tube_values],
                                        mode='lines+markers',
                                        name=f'Part {part}',
                                        line=dict(width=2)))

        # Mise en forme du titre et des axes pour chaque figure

        fig_top.update_layout(title=f'Qdot_top - Cas {case+1} - Simu M{mesh+1}',
                            xaxis_title='Itération',
                            yaxis_title='Qdot_top')

        # Affichage des figures
        fig_top.show()

In [ ]:
## Affichage Qdot_tube // CFD

for mesh in range(nb_mesh):
    # Boucle sur chaque cas
    for case in range(nb_cases):
        # Listes pour stocker les valeurs de chaque transfert pour chaque itération
        Qdot_tube_values = []

        # Boucle sur chaque itération pour calculer les valeurs
        for iteration in range(limit_big_it):
            Qdot_tube, Qdot_top, Qdot_abs, Qdot_top_rad = calculate_Qdot(mesh, case, iteration)
            Qdot_tube_values.append(Qdot_tube)
        
        # Création des figures pour chaque type de transfert
        fig_tube = go.Figure()

        # Ajout des traces à chaque figure
        for part in {3,5}:
            fig_tube.add_trace(go.Scatter(x=list(range(1, limit_big_it + 1)), y=[qt[part-1] for qt in Qdot_tube_values],
                                        mode='lines+markers',
                                        name=f'Part {part}',
                                        line=dict(width=2)))

        # Mise en forme du titre et des axes pour chaque figure
        fig_tube.update_layout(title=f'Qdot_tube - Cas {case+1} - Simu M{mesh+1}',
                            xaxis_title='Itération',
                            yaxis_title='Qdot_tube')

        # Affichage des figures
        fig_tube.show()


CFD et 1D sur une même courbe

In [ ]:
## Affichage Qdot_tube // CFD et 1D

# Initialiser les couleurs pour les parties (7 couleurs différentes)
part_colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2']

for mesh in range(nb_mesh):
    # Boucle sur chaque cas
    for case in range(nb_cases):
        # Listes pour stocker les valeurs de chaque transfert pour chaque itération pour CFD et 1D
        Qdot_tube_values_cfd = []
        Qdot_tube_values_1d = [[] for _ in range(nb_hx + 3)]
        Qdot_tube_values = [[] for _ in range(nb_hx + 3)]
        iterations_cfd = []
        iterations_1d = [[] for _ in range(nb_hx + 3)]
        iterations = [[] for _ in range(nb_hx + 3)]

        # Créer une figure pour les valeurs de Qdot_top
        fig_top = go.Figure()

        # Boucle sur chaque itération pour calculer les valeurs et les afficher
        for iteration in range(limit_big_it):
            # Calculer les valeurs CFD
            Qdot_tube, Qdot_top, Qdot_abs, Qdot_top_rad = calculate_Qdot(mesh, case, iteration)
            Qdot_tube_values_cfd.append(Qdot_tube)
            iterations_cfd.append(iteration + 1)

            # Récupérer les valeurs pour Qdot_top_conv pour 1D
            for part in {3,5}:
                tube_value_1d = df_one_mesh_case_list[mesh][case][iteration]['Qdot_tube_back'].loc[f'part{part}']
                Qdot_tube_values_1d[part-1].append(tube_value_1d)
                iterations_1d[part-1].append(iteration + 1.5)
                Qdot_tube_values[part-1].append(Qdot_tube[part-1])
                Qdot_tube_values[part-1].append(tube_value_1d)
                iterations[part-1].append(iteration + 1)
                iterations[part-1].append(iteration + 1.5)

        # Ajouter les valeurs CFD et 1D à la figure
        for part in range(1, nb_hx + 3):
            # Tracer les valeurs CFD
            fig_top.add_trace(go.Scatter(x=iterations_cfd, 
                                        y=[qt[part-1] for qt in Qdot_tube_values_cfd],
                                        mode='markers',
                                        name=f'Part {part} CFD',
                                        line=dict(width=2, color=part_colors[part-1]),
                                        marker=dict(symbol='cross')))

            # Tracer les valeurs 1D
            fig_top.add_trace(go.Scatter(x=iterations_1d[part-1], 
                                        y=Qdot_tube_values_1d[part-1],
                                        mode='markers',
                                        name=f'Part {part} 1D',
                                        line=dict(width=2, color=part_colors[part-1], dash='dot'),
                                        marker=dict(symbol='x')))
            
            fig_top.add_trace(go.Scatter(x=iterations[part-1], 
                                        y=Qdot_tube_values[part-1],
                                        mode='lines',
                                        name=f'Part {part}',
                                        line=dict(width=2, color=part_colors[part-1], dash='dot')))

        # Mise en forme du titre et des axes
        fig_top.update_layout(title=f'Qdot_tube - Cas {case+1} - Simu M{mesh+1}',
                            xaxis_title='Itération',
                            yaxis_title='Qdot_tube_back')

        # Affichage de la figure
        fig_top.show()

In [ ]:
## Affichage Qdot_top // CFD et 1D

# Initialiser les couleurs pour les parties (7 couleurs différentes)
part_colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2']

for mesh in range(nb_mesh):
    # Boucle sur chaque cas
    for case in range(nb_cases):
        # Listes pour stocker les valeurs de chaque transfert pour chaque itération pour CFD et 1D
        Qdot_top_values_cfd = []
        Qdot_top_values_1d = [[] for _ in range(nb_hx + 3)]
        Qdot_top_values = [[] for _ in range(nb_hx + 3)]
        iterations_cfd = []
        iterations_1d = [[] for _ in range(nb_hx + 3)]
        iterations = [[] for _ in range(nb_hx + 3)]

        # Créer une figure pour les valeurs de Qdot_top
        fig_top = go.Figure()

        # Boucle sur chaque itération pour calculer les valeurs et les afficher
        for iteration in range(limit_big_it):
            # Calculer les valeurs CFD
            Qdot_tube, Qdot_top, Qdot_abs, Qdot_top_rad = calculate_Qdot(mesh, case, iteration)
            Qdot_top_values_cfd.append(Qdot_top)
            iterations_cfd.append(iteration + 1)

            # Récupérer les valeurs pour Qdot_top_conv pour 1D
            for part in range(1, nb_hx + 3):
                top_value_1d = df_one_mesh_case_list[mesh][case][iteration]['Qdot_top_conv'].loc[f'part{part}']
                Qdot_top_values_1d[part-1].append(top_value_1d)
                iterations_1d[part-1].append(iteration + 1.5)
                Qdot_top_values[part-1].append(Qdot_top[part-1])
                Qdot_top_values[part-1].append(top_value_1d)
                iterations[part-1].append(iteration + 1)
                iterations[part-1].append(iteration + 1.5)

        # Ajouter les valeurs CFD et 1D à la figure
        for part in range(1, nb_hx + 3):
            # Tracer les valeurs CFD
            fig_top.add_trace(go.Scatter(x=iterations_cfd, 
                                        y=[qt[part-1] for qt in Qdot_top_values_cfd],
                                        mode='markers',
                                        name=f'Part {part} CFD',
                                        line=dict(width=2, color=part_colors[part-1]),
                                        marker=dict(symbol='cross')))

            # Tracer les valeurs 1D
            fig_top.add_trace(go.Scatter(x=iterations_1d[part-1], 
                                        y=Qdot_top_values_1d[part-1],
                                        mode='markers',
                                        name=f'Part {part} 1D',
                                        line=dict(width=2, color=part_colors[part-1], dash='dot'),
                                        marker=dict(symbol='x')))
            
            fig_top.add_trace(go.Scatter(x=iterations[part-1], 
                                        y=Qdot_top_values[part-1],
                                        mode='lines',
                                        name=f'Part {part}',
                                        line=dict(width=2, color=part_colors[part-1], dash='dot')))

        # Mise en forme du titre et des axes
        fig_top.update_layout(title=f'Qdot_top - Cas {case+1} - Simu M{mesh+1}',
                            xaxis_title='Itération',
                            yaxis_title='Qdot_top')

        # Affichage de la figure
        fig_top.show()


Comparaison des maillages


In [ ]:
## Comparaison Qdot_tube_back_conv pour chaque partie // 1D

# Initialisation de la liste de figures pour les graphiques par cas
fig_comparison_list = []

# Couleurs pour chaque simulation (choisissez autant de couleurs que de simulations)
sim_colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2']

# Largeur des barres pour chaque simulation
bar_width = 0.8 / nb_mesh

# Positions des barres pour chaque partie et chaque simulation
bar_positions = np.arange(1, nb_hx + 4)  # positions pour chaque partie (1 à nb_hx + 3)

# Boucle sur chaque cas
for case in range(nb_cases):
    # Créer une figure pour comparer les simulations
    fig_comparison = go.Figure()

    # Pour chaque simulation (chaque maillage)
    for mesh in range(nb_mesh):
        # Liste pour stocker les valeurs de Qdot_tube_back_conv pour chaque partie
        tube_values = []

        # Pour chaque partie
        for part in {3,5}:
            # Récupérer les valeurs de Qdot_tube_back_conv pour la dernière itération pour ce cas et ce maillage
            iteration = limit_big_it - 1
            tube_value = df_one_mesh_case_list[mesh][case][iteration]['Qdot_tube_back_conv'].loc[f'part{part}']
            tube_values.append(tube_value)

        # Ajouter les barres pour chaque partie
        fig_comparison.add_trace(go.Bar(
            x=bar_positions + mesh * bar_width - 0.4,
            y=tube_values,
            name=f'Simulation M{mesh + 1}',
            marker_color=sim_colors[mesh],
            width=bar_width,
            opacity=0.8
        ))

    # Mise en forme du titre et des axes
    fig_comparison.update_layout(
        title=f'Comparaison des simulations pour Qdot_tube_back_conv - Cas {case + 1}',
        xaxis_title='Parties',
        yaxis_title='Qdot_tube_back_conv',
        barmode='group',  # Regrouper les barres
        legend_title='Simulation'
    )

    # Ajouter la figure à la liste
    fig_comparison_list.append(fig_comparison)

# Afficher toutes les figures
for fig_comparison in fig_comparison_list:
    fig_comparison.show()


In [ ]:
## Comparaison Qdot_top_conv pour chaque partie // 1D

# Initialisation de la liste de figures pour les graphiques par cas
fig_comparison_list = []

# Couleurs pour chaque simulation (choisissez autant de couleurs que de simulations)
sim_colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2']

# Largeur des barres pour chaque simulation
bar_width = 0.8 / nb_mesh

# Positions des barres pour chaque partie et chaque simulation
bar_positions = np.arange(1, nb_hx + 4)  # positions pour chaque partie (1 à nb_hx + 3)

# Boucle sur chaque cas
for case in range(nb_cases):
    # Créer une figure pour comparer les simulations
    fig_comparison = go.Figure()

    # Pour chaque simulation (chaque maillage)
    for mesh in range(nb_mesh):
        # Liste pour stocker les valeurs de Qdot_tube_back_conv pour chaque partie
        tube_values = []

        # Pour chaque partie
        for part in range(1, nb_hx + 3):
            # Récupérer les valeurs de Qdot_tube_back_conv pour la dernière itération pour ce cas et ce maillage
            iteration = limit_big_it - 1
            tube_value = df_one_mesh_case_list[mesh][case][iteration]['Qdot_top_conv'].loc[f'part{part}']
            tube_values.append(tube_value)

        # Ajouter les barres pour chaque partie
        fig_comparison.add_trace(go.Bar(
            x=bar_positions + mesh * bar_width - 0.4,
            y=tube_values,
            name=f'Simulation M{mesh + 1}',
            marker_color=sim_colors[mesh],
            width=bar_width,
            opacity=0.8
        ))

    # Mise en forme du titre et des axes
    fig_comparison.update_layout(
        title=f'Comparaison des simulations pour Qdot_top_conv - Cas {case + 1}',
        xaxis_title='Parties',
        yaxis_title='Qdot_tube_back_conv',
        barmode='group',  # Regrouper les barres
        legend_title='Simulation'
    )

    # Ajouter la figure à la liste
    fig_comparison_list.append(fig_comparison)

# Afficher toutes les figures
for fig_comparison in fig_comparison_list:
    fig_comparison.show()


In [ ]:
## Comparaison Qdot_top_rad pour chaque partie // 1D

# Initialisation de la liste de figures pour les graphiques par cas
fig_comparison_list = []

# Couleurs pour chaque simulation (choisissez autant de couleurs que de simulations)
sim_colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2']

# Largeur des barres pour chaque simulation
bar_width = 0.8 / nb_mesh

# Positions des barres pour chaque partie et chaque simulation
bar_positions = np.arange(1, nb_hx + 4)  # positions pour chaque partie (1 à nb_hx + 3)

# Boucle sur chaque cas
for case in range(nb_cases):
    # Créer une figure pour comparer les simulations
    fig_comparison = go.Figure()

    # Pour chaque simulation (chaque maillage)
    for mesh in range(nb_mesh):
        # Liste pour stocker les valeurs de Qdot_tube_back_conv pour chaque partie
        tube_values = []

        # Pour chaque partie
        for part in range(1, nb_hx + 3):
            # Récupérer les valeurs de Qdot_tube_back_conv pour la dernière itération pour ce cas et ce maillage
            iteration = limit_big_it - 1
            tube_value = df_one_mesh_case_list[mesh][case][iteration]['Qdot_top_rad'].loc[f'part{part}']
            tube_values.append(tube_value)

        # Ajouter les barres pour chaque partie
        fig_comparison.add_trace(go.Bar(
            x=bar_positions + mesh * bar_width - 0.4,
            y=tube_values,
            name=f'Simulation M{mesh + 1}',
            marker_color=sim_colors[mesh],
            width=bar_width,
            opacity=0.8
        ))

    # Mise en forme du titre et des axes
    fig_comparison.update_layout(
        title=f'Comparaison des simulations pour Qdot_top_rad - Cas {case + 1}',
        xaxis_title='Parties',
        yaxis_title='Qdot_tube_back_conv',
        barmode='group',  # Regrouper les barres
        legend_title='Simulation'
    )

    # Ajouter la figure à la liste
    fig_comparison_list.append(fig_comparison)

# Afficher toutes les figures
for fig_comparison in fig_comparison_list:
    fig_comparison.show()


In [ ]:
## Comparaison Qdot_top_rad pour chaque partie // CFD

# Couleurs pour chaque simulation (autant de couleurs que de simulations)
sim_colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2']

# Initialisation de la liste de figures pour les graphiques par cas
fig_comparison_list = []

# Largeur des barres pour chaque simulation
bar_width = 0.8 / nb_mesh

# Positions des barres pour chaque partie et chaque simulation
bar_positions = np.arange(1, nb_hx + 3)  # positions pour chaque partie (1 à nb_hx)

# Boucle sur chaque cas
for case in range(nb_cases):
    # Créer une figure pour comparer les simulations
    fig_comparison = go.Figure()

    # Pour chaque simulation (chaque maillage)
    for mesh in range(nb_mesh):
        # Liste pour stocker les valeurs de Qdot_top_rad pour chaque partie
        top_rad_values = []

        # Pour chaque partie
        for part in range(1, nb_hx + 3):
            # Récupérer les valeurs de Qdot_top_rad pour la dernière itération pour ce cas et ce maillage
            iteration = limit_big_it - 1
            Qdot_tube, Qdot_top, Qdot_abs, Qdot_top_rad = calculate_Qdot(mesh, case, iteration)
            top_rad_values.append(Qdot_top_rad[part - 1])

        # Ajouter les barres pour chaque partie
        fig_comparison.add_trace(go.Bar(
            x=bar_positions + (mesh - (nb_mesh - 1) / 2) * bar_width,
            y=top_rad_values,
            name=f'Simulation M{mesh + 1}',
            marker_color=sim_colors[mesh],
            width=bar_width,
            opacity=0.8
        ))

    # Mise en forme du titre et des axes
    fig_comparison.update_layout(
        title=f'Comparaison des simulations pour Qdot_top_rad - Cas {case + 1}',
        xaxis_title='Parties',
        yaxis_title='Qdot_top_rad',
        xaxis=dict(
            tickmode='array',
            tickvals=np.arange(1, nb_hx + 3),
            ticktext=[f'Part {i}' for i in range(1, nb_hx + 3)]
        ),
        barmode='group',  # Regrouper les barres
        legend_title='Simulation'
    )

    # Ajouter la figure à la liste
    fig_comparison_list.append(fig_comparison)

# Afficher toutes les figures
for fig_comparison in fig_comparison_list:
    fig_comparison.show()


In [ ]:
## Comparaison Qdot_top_conv pour chaque partie // CFD

# Couleurs pour chaque simulation (autant de couleurs que de simulations)
sim_colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2']

# Initialisation de la liste de figures pour les graphiques par cas
fig_comparison_list = []

# Largeur des barres pour chaque simulation
bar_width = 0.8 / nb_mesh

# Positions des barres pour chaque partie et chaque simulation
bar_positions = np.arange(1, nb_hx + 3)  # positions pour chaque partie (1 à nb_hx)

# Boucle sur chaque cas
for case in range(nb_cases):
    # Créer une figure pour comparer les simulations
    fig_comparison = go.Figure()

    # Pour chaque simulation (chaque maillage)
    for mesh in range(nb_mesh):
        # Liste pour stocker les valeurs de Qdot_top_rad pour chaque partie
        top_rad_values = []

        # Pour chaque partie
        for part in range(1, nb_hx + 3):
            # Récupérer les valeurs de Qdot_top_rad pour la dernière itération pour ce cas et ce maillage
            iteration = limit_big_it - 1
            Qdot_tube, Qdot_top, Qdot_abs, Qdot_top_rad = calculate_Qdot(mesh, case, iteration)
            top_rad_values.append(Qdot_top[part - 1])

        # Ajouter les barres pour chaque partie
        fig_comparison.add_trace(go.Bar(
            x=bar_positions + (mesh - (nb_mesh - 1) / 2) * bar_width,
            y=top_rad_values,
            name=f'Simulation M{mesh + 1}',
            marker_color=sim_colors[mesh],
            width=bar_width,
            opacity=0.8
        ))

    # Mise en forme du titre et des axes
    fig_comparison.update_layout(
        title=f'Comparaison des simulations pour Qdot_top - Cas {case + 1}',
        xaxis_title='Parties',
        yaxis_title='Qdot_top',
        xaxis=dict(
            tickmode='array',
            tickvals=np.arange(1, nb_hx + 3),
            ticktext=[f'Part {i}' for i in range(1, nb_hx + 3)]
        ),
        barmode='group',  # Regrouper les barres
        legend_title='Simulation'
    )

    # Ajouter la figure à la liste
    fig_comparison_list.append(fig_comparison)

# Afficher toutes les figures
for fig_comparison in fig_comparison_list:
    fig_comparison.show()


In [ ]:
## Comparaison Qdot_tube pour chaque partie // CFD

# Couleurs pour chaque simulation (autant de couleurs que de simulations)
sim_colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2']

# Initialisation de la liste de figures pour les graphiques par cas
fig_comparison_list = []

# Largeur des barres pour chaque simulation
bar_width = 0.8 / nb_mesh

# Positions des barres pour chaque partie et chaque simulation
bar_positions = np.arange(1, nb_hx + 3)  # positions pour chaque partie (1 à nb_hx)

# Boucle sur chaque cas
for case in range(nb_cases):
    # Créer une figure pour comparer les simulations
    fig_comparison = go.Figure()

    # Pour chaque simulation (chaque maillage)
    for mesh in range(nb_mesh):
        # Liste pour stocker les valeurs de Qdot_top_rad pour chaque partie
        top_rad_values = []

        # Pour chaque partie
        for part in range(1, nb_hx + 3):
            # Récupérer les valeurs de Qdot_top_rad pour la dernière itération pour ce cas et ce maillage
            iteration = limit_big_it - 1
            Qdot_tube, Qdot_top, Qdot_abs, Qdot_top_rad = calculate_Qdot(mesh, case, iteration)
            top_rad_values.append(Qdot_tube[part - 1])

        # Ajouter les barres pour chaque partie
        fig_comparison.add_trace(go.Bar(
            x=bar_positions + (mesh - (nb_mesh - 1) / 2) * bar_width,
            y=top_rad_values,
            name=f'Simulation M{mesh + 1}',
            marker_color=sim_colors[mesh],
            width=bar_width,
            opacity=0.8
        ))

    # Mise en forme du titre et des axes
    fig_comparison.update_layout(
        title=f'Comparaison des simulations pour Qdot_tube - Cas {case + 1}',
        xaxis_title='Parties',
        yaxis_title='Qdot_tube',
        xaxis=dict(
            tickmode='array',
            tickvals=np.arange(1, nb_hx + 3),
            ticktext=[f'Part {i}' for i in range(1, nb_hx + 3)]
        ),
        barmode='group',  # Regrouper les barres
        legend_title='Simulation'
    )

    # Ajouter la figure à la liste
    fig_comparison_list.append(fig_comparison)

# Afficher toutes les figures
for fig_comparison in fig_comparison_list:
    fig_comparison.show()


In [ ]:
## Comparaison de la convergence Qdot_top_conv // CFD et 1D 

# Initialiser les couleurs pour les parties (7 couleurs différentes)
part_colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2']

# Boucle sur chaque cas
for case in range(nb_cases):
    # Créer une figure pour les valeurs de Qdot_tube
    fig_top = go.Figure()

    # Pour chaque simulation (chaque maillage)
    for mesh in range(nb_mesh):
        # Listes pour stocker les valeurs de chaque transfert pour chaque itération pour CFD et 1D
        Qdot_top_values_cfd = []
        Qdot_top_values_1d = [[] for _ in range(nb_hx + 3)]
        Qdot_combined_values = [[] for _ in range(nb_hx + 3)]
        iterations_cfd = []
        iterations_1d = [[] for _ in range(nb_hx + 3)]
        iterations_combined = [[] for _ in range(nb_hx + 3)]

        # Boucle sur chaque itération pour calculer les valeurs
        for iteration in range(limit_big_it):
            # Calculer les valeurs CFD
            Qdot_tube, Qdot_top, Qdot_abs, Qdot_top_rad = calculate_Qdot(mesh, case, iteration)
            Qdot_top_values_cfd.append(Qdot_top)
            iterations_cfd.append(iteration + 1)

            # Récupérer les valeurs pour Qdot_tube_back pour 1D
            for part in range(1, nb_hx + 3):
                top_value_1d = df_one_mesh_case_list[mesh][case][iteration]['Qdot_top_conv'].loc[f'part{part}']
                Qdot_top_values_1d[part-1].append(top_value_1d)
                iterations_1d[part-1].append(iteration + 1.5)

                # Combinaison des valeurs CFD et 1D
                Qdot_combined_values[part-1].append(Qdot_top[part-1])
                Qdot_combined_values[part-1].append(top_value_1d)
                iterations_combined[part-1].append(iteration + 1)
                iterations_combined[part-1].append(iteration + 1.5)

        # Ajouter les valeurs CFD et 1D à la figure
        for part in range(1, nb_hx + 3):
            # Tracer les valeurs CFD
            fig_top.add_trace(go.Scatter(x=iterations_cfd, 
                                          y=[qt[part-1] for qt in Qdot_top_values_cfd],
                                          mode='markers',
                                          name=f'Part {part} CFD - Simu M{mesh+1}',
                                          line=dict(width=2, color=part_colors[part-1]),
                                          marker=dict(symbol='cross')))

            # Tracer les valeurs 1D
            fig_top.add_trace(go.Scatter(x=iterations_1d[part-1], 
                                          y=Qdot_top_values_1d[part-1],
                                          mode='markers',
                                          name=f'Part {part} 1D - Simu M{mesh+1}',
                                          line=dict(width=2, color=part_colors[part-1], dash='dot'),
                                          marker=dict(symbol='x')))
            
            # Tracer la courbe combinée
            fig_top.add_trace(go.Scatter(x=iterations_combined[part-1], 
                                          y=Qdot_combined_values[part-1],
                                          mode='lines',
                                          name=f'Part {part} Combiné',
                                          line=dict(width=2, color=part_colors[part-1], dash ='dot')))

    # Mise en forme du titre et des axes
    fig_top.update_layout(title=f'Qdot_top_conv - Cas {case+1}',
                           xaxis_title='Itération',
                           yaxis_title='Qdot_tube_back',
                           legend_title='Partie et Simulation')

    # Affichage de la figure
    fig_top.show()


In [ ]:
## Comparaison Qdot_tube pour chaque partie // CFD et 1D

# Initialiser les couleurs pour les parties (7 couleurs différentes)
part_colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2']

# Boucle sur chaque cas
for case in range(nb_cases):
    # Créer une figure pour les valeurs de Qdot_tube
    fig_tube = go.Figure()

    # Pour chaque simulation (chaque maillage)
    for mesh in range(nb_mesh):
        # Listes pour stocker les valeurs de chaque transfert pour chaque itération pour CFD et 1D
        Qdot_tube_values_cfd = []
        Qdot_tube_values_1d = [[] for _ in range(nb_hx + 3)]
        Qdot_combined_values = [[] for _ in range(nb_hx + 3)]
        iterations_cfd = []
        iterations_1d = [[] for _ in range(nb_hx + 3)]
        iterations_combined = [[] for _ in range(nb_hx + 3)]

        # Boucle sur chaque itération pour calculer les valeurs
        for iteration in range(limit_big_it):
            # Calculer les valeurs CFD
            Qdot_tube, Qdot_top, Qdot_abs, Qdot_top_rad = calculate_Qdot(mesh, case, iteration)
            Qdot_tube_values_cfd.append(Qdot_tube)
            iterations_cfd.append(iteration + 1)

            # Récupérer les valeurs pour Qdot_tube_back pour 1D
            for part in {3,5}:
                tube_value_1d = df_one_mesh_case_list[mesh][case][iteration]['Qdot_tube_back'].loc[f'part{part}']
                Qdot_tube_values_1d[part-1].append(tube_value_1d)
                iterations_1d[part-1].append(iteration + 1.5)

                # Combinaison des valeurs CFD et 1D
                Qdot_combined_values[part-1].append(Qdot_tube[part-1])
                Qdot_combined_values[part-1].append(tube_value_1d)
                iterations_combined[part-1].append(iteration + 1)
                iterations_combined[part-1].append(iteration + 1.5)

        # Ajouter les valeurs CFD et 1D à la figure
        for part in {3,5}:
            # Tracer les valeurs CFD
            fig_tube.add_trace(go.Scatter(x=iterations_cfd, 
                                          y=[qt[part-1] for qt in Qdot_tube_values_cfd],
                                          mode='markers',
                                          name=f'Simu M{mesh+1}-Part{part} CFD',
                                          line=dict(width=2, color=part_colors[part-1]),
                                          marker=dict(symbol='cross')))

            # Tracer les valeurs 1D
            fig_tube.add_trace(go.Scatter(x=iterations_1d[part-1], 
                                          y=Qdot_tube_values_1d[part-1],
                                          mode='markers',
                                          name=f'Simu M{mesh+1}-Part{part} 1D',
                                          line=dict(width=2, color=part_colors[part-1], dash='dot'),
                                          marker=dict(symbol='x')))
            
            # Tracer la courbe combinée
            fig_tube.add_trace(go.Scatter(x=iterations_combined[part-1], 
                                          y=Qdot_combined_values[part-1],
                                          mode='lines',
                                          name=f'Part {part} Combiné',
                                          line=dict(width=2, color=part_colors[part-1], dash ='dot')))

    # Mise en forme du titre et des axes
    fig_tube.update_layout(title=f'Qdot_tube - Cas {case+1}',
                           xaxis_title='Itération',
                           yaxis_title='Qdot_tube_back',
                           legend_title='Partie et Simulation')

    # Affichage de la figure
    fig_tube.show()


### Cas par cas

In [94]:
### Comparaison cas par cas

# Définir le cas que vous voulez afficher
case = 0

# Affichage Qdot_top // CFD et 1D
# Initialiser les couleurs pour les parties (7 couleurs différentes)
part_colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2']
# Couleurs pour chaque simulation (choisissez autant de couleurs que de simulations)
sim_colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2']
bar_width = 0.8 / nb_mesh
bar_positions = np.arange(1, nb_hx + 4)  # positions pour chaque partie (1 à nb_hx + 3)


## Convergence de Qdot_top pour chaque partie // CFD et 1D

fig_top = go.Figure()
# Pour chaque simulation (chaque maillage)
for mesh in range(nb_mesh):
    # Listes pour stocker les valeurs de chaque transfert pour chaque itération pour CFD et 1D
    Qdot_top_values_cfd = []
    Qdot_top_values_1d = [[] for _ in range(nb_hx + 3)]
    Qdot_combined_values = [[] for _ in range(nb_hx + 3)]
    iterations_cfd = []
    iterations_1d = [[] for _ in range(nb_hx + 3)]
    iterations_combined = [[] for _ in range(nb_hx + 3)]

    # Boucle sur chaque itération pour calculer les valeurs
    for iteration in range(limit_big_it):
        # Calculer les valeurs CFD
        Qdot_tube, Qdot_top, Qdot_abs, Qdot_top_rad = calculate_Qdot(mesh, case, iteration)
        Qdot_top_values_cfd.append(Qdot_top)
        iterations_cfd.append(iteration + 1)

        # Récupérer les valeurs pour Qdot_top pour 1D
        # for part in range(1, nb_hx + 3):
        for part in {3,5}:
            top_value_1d = df_one_mesh_case_list[mesh][case][iteration]['Qdot_top_conv'].loc[f'part{part}']
            Qdot_top_values_1d[part-1].append(top_value_1d)
            iterations_1d[part-1].append(iteration + 1.5)

            # Combinaison des valeurs CFD et 1D
            Qdot_combined_values[part-1].append(Qdot_top[part-1])
            Qdot_combined_values[part-1].append(top_value_1d)
            iterations_combined[part-1].append(iteration + 1)
            iterations_combined[part-1].append(iteration + 1.5)

    # Ajouter les valeurs CFD et 1D à la figure
    # for part in range(1, nb_hx + 3):
    for part in {3,5}:
        # Tracer les valeurs CFD
        fig_top.add_trace(go.Scatter(x=iterations_cfd, 
                                        y=[qt[part-1] for qt in Qdot_top_values_cfd],
                                        mode='markers',
                                        name='Qdot_top CFD',
                                        line=dict(width=2, color=part_colors[part-1]),
                                        marker=dict(symbol='cross'),
                                        hovertemplate= f'Simu M{mesh+1} <br>Part{part} <br>'+'%{y:.2f}'))

        # Tracer les valeurs 1D
        fig_top.add_trace(go.Scatter(x=iterations_1d[part-1], 
                                        y=Qdot_top_values_1d[part-1],
                                        mode='markers',
                                        name='Qdot_top_conv 1D',
                                        line=dict(width=2, color=part_colors[part-1], dash='dot'),
                                        marker=dict(symbol='x'),
                                        hovertemplate= f'Simu M{mesh+1} <br>Part{part} <br>'+'%{y:.2f}'))
        
        # Tracer la courbe combinée
        fig_top.add_trace(go.Scatter(x=iterations_combined[part-1], 
                                        y=Qdot_combined_values[part-1],
                                        mode='lines',
                                        name=f'Part {part}',
                                        line=dict(width=2, color=part_colors[part-1], dash ='dot')))

# Mise en forme du titre et des axes
fig_top.update_layout(title=f'Qdot_top - Cas {case+1}',
                        xaxis_title='Itération',
                        yaxis_title='Qdot_top [W]',
                        legend_title='Partie et Simulation')

# Affichage de la figure
fig_top.show()





## Convergence de Qdot_tube_back pour chaque partie // CFD et 1D

fig_tube = go.Figure()
# Pour chaque simulation (chaque maillage)
for mesh in range(nb_mesh):
    # Listes pour stocker les valeurs de chaque transfert pour chaque itération pour CFD et 1D
    Qdot_tube_values_cfd = []
    Qdot_tube_values_1d = [[] for _ in range(nb_hx + 3)]
    Qdot_combined_values = [[] for _ in range(nb_hx + 3)]
    iterations_cfd = []
    iterations_1d = [[] for _ in range(nb_hx + 3)]
    iterations_combined = [[] for _ in range(nb_hx + 3)]

    # Boucle sur chaque itération pour calculer les valeurs
    for iteration in range(limit_big_it):
        # Calculer les valeurs CFD
        Qdot_tube, Qdot_top, Qdot_abs, Qdot_top_rad = calculate_Qdot(mesh, case, iteration)
        Qdot_tube_values_cfd.append(Qdot_tube)
        iterations_cfd.append(iteration + 1)

        # Récupérer les valeurs pour Qdot_tube_back pour 1D
        for part in {3,5}:
            tube_value_1d = df_one_mesh_case_list[mesh][case][iteration]['Qdot_tube_back'].loc[f'part{part}']
            Qdot_tube_values_1d[part-1].append(tube_value_1d)
            iterations_1d[part-1].append(iteration + 1.5)

            # Combinaison des valeurs CFD et 1D
            Qdot_combined_values[part-1].append(Qdot_tube[part-1])
            Qdot_combined_values[part-1].append(tube_value_1d)
            iterations_combined[part-1].append(iteration + 1)
            iterations_combined[part-1].append(iteration + 1.5)

    # Ajouter les valeurs CFD et 1D à la figure
    for part in {3,5}:
        # Tracer les valeurs CFD
        fig_tube.add_trace(go.Scatter(x=iterations_cfd, 
                                        y=[qt[part-1] for qt in Qdot_tube_values_cfd],
                                        mode='markers',
                                        name='Qdot_tube CFD',
                                        line=dict(width=2, color=part_colors[part-1]),
                                        marker=dict(symbol='cross'),
                                        hovertemplate= f'Simu M{mesh+1} <br>Part{part} <br>'+'%{y:.2f}'))

        # Tracer les valeurs 1D
        fig_tube.add_trace(go.Scatter(x=iterations_1d[part-1], 
                                        y=Qdot_tube_values_1d[part-1],
                                        mode='markers',
                                        name='Qdot_tube_back 1D',
                                        line=dict(width=2, color=part_colors[part-1], dash='dot'),
                                        marker=dict(symbol='x'),
                                        hovertemplate= f'Simu M{mesh+1} <br>Part{part} <br>'+'%{y:.2f}'))
        
        # Tracer la courbe combinée
        fig_tube.add_trace(go.Scatter(x=iterations_combined[part-1], 
                                        y=Qdot_combined_values[part-1],
                                        mode='lines',
                                        name=f'Part {part}',
                                        line=dict(width=2, color=part_colors[part-1], dash ='dot')))

# Mise en forme du titre et des axes
fig_tube.update_layout(title=f'Qdot_tube_back - Cas {case+1}',
                        xaxis_title='Itération',
                        yaxis_title='Qdot_tube_back [W]',
                        legend_title='Partie et Simulation')

# Affichage de la figure
fig_tube.show()





## Comparaison des maillages pour Qdot_tube_fluid // 1D
# Créer une figure pour comparer les simulations
fig_comparison = go.Figure()

# Pour chaque simulation (chaque maillage)
for mesh in range(nb_mesh):
    # Liste pour stocker les valeurs de Qdot_tube_fluid pour chaque partie
    tube_values = []

    # Pour chaque partie
    for part in range(1, nb_hx + 3):
        # Récupérer les valeurs de Qdot_tube_fluid pour la dernière itération pour ce cas et ce maillage
        iteration = limit_big_it - 1
        tube_value = df_one_mesh_case_list[mesh][case][iteration]['Qdot_tube_fluid'].loc[f'part{part}']
        tube_values.append(tube_value)

    # Ajouter les barres pour chaque partie
    fig_comparison.add_trace(go.Bar(
        x=bar_positions + mesh * bar_width - 0.4,
        y=tube_values,
        name=f'Qdot_tube_fluid 1D',
        marker_color=sim_colors[mesh],
        width=bar_width,
        opacity=0.8,
        hovertemplate= f'Simu M{mesh+1} <br>Part{part} <br>'+'%{y:.2f}'
    ))

# Mise en forme du titre et des axes
fig_comparison.update_layout(
    title=f'Qdot_tube_fluid - Cas {case + 1} - 1D',
    xaxis_title='Parties',
    yaxis_title='Qdot_tube_fluid [W]',
    barmode='group',  # Regrouper les barres
    legend_title='Simulation'
)

# Affichage de la figure
fig_comparison.show()



## Comparaison des maillages pour Qdot_top_conv // 1D
# Via ratio
# Créer une figure pour comparer les simulations
fig_comparison = go.Figure()

# Pour chaque simulation (chaque maillage)
for mesh in range(nb_mesh):
    # Liste pour stocker les valeurs de Qdot_top_conv pour chaque partie
    top_values = []

    # Pour chaque partie
    for part in range(1, nb_hx + 3):
        # Récupérer les valeurs de Qdot_top_conv pour la dernière itération pour ce cas et ce maillage
        iteration = limit_big_it - 1
        ratio  = ratio_top_list[mesh][case][iteration][part-1][0]
        top_value = ratio*df_one_mesh_case_list[mesh][case][iteration]['Qdot_top_conv'].loc[f'part{part}']
        top_values.append(top_value)

    # Ajouter les barres pour chaque partie
    fig_comparison.add_trace(go.Bar(
        x=bar_positions + mesh * bar_width - 0.4,
        y=top_values,
        name=f'Qdot_top_conv 1D',
        marker_color=sim_colors[mesh],
        width=bar_width,
        opacity=0.8,
        hovertemplate= f'Simu M{mesh+1} <br>Part{part} <br>'+'%{y:.2f}'
    ))

# Mise en forme du titre et des axes
fig_comparison.update_layout(
    title=f'Qdot_top_conv - Cas {case + 1} - 1D',
    xaxis_title='Parties',
    yaxis_title='Qdot_top_conv [W]',
    barmode='group',  # Regrouper les barres
    legend_title='Simulation'
)

# Affichage de la figure
fig_comparison.show()



## Comparaison des maillages pour Qdot_top_rad // 1D
# Via ratio
# Créer une figure pour comparer les simulations
fig_comparison = go.Figure()

# Pour chaque simulation (chaque maillage)
for mesh in range(nb_mesh):
    # Liste pour stocker les valeurs de Qdot_top_rad pour chaque partie
    top_values = []

    # Pour chaque partie
    for part in range(1, nb_hx + 3):
        # Récupérer les valeurs de Qdot_top_rad pour la dernière itération pour ce cas et ce maillage
        iteration = limit_big_it - 1
        ratio  = ratio_top_list[mesh][case][iteration][part-1][1]
        top_value = ratio*df_one_mesh_case_list[mesh][case][iteration]['Qdot_top_conv'].loc[f'part{part}']
        top_values.append(top_value)

    # Ajouter les barres pour chaque partie
    fig_comparison.add_trace(go.Bar(
        x=bar_positions + mesh * bar_width - 0.4,
        y=top_values,
        name=f'Qdot_top_rad 1D',
        marker_color=sim_colors[mesh],
        width=bar_width,
        opacity=0.8,
        hovertemplate= f'Simu M{mesh+1} <br>Part{part} <br>'+'%{y:.2f}'
    ))

# Mise en forme du titre et des axes
fig_comparison.update_layout(
    title=f'Qdot_top_rad - Cas {case + 1} - 1D',
    xaxis_title='Parties',
    yaxis_title='Qdot_top_rad [W]',
    barmode='group',  # Regrouper les barres
    legend_title='Simulation'
)

# Affichage de la figure
fig_comparison.show()



## Comparaison des maillages pour Qdot_tube_back_conv // 1D
# Via ratio
# Créer une figure pour comparer les simulations
fig_comparison = go.Figure()

# Pour chaque simulation (chaque maillage)
for mesh in range(nb_mesh):
    # Liste pour stocker les valeurs de Qdot_tube_back_conv pour chaque partie
    tube_back_values = []

    # Pour chaque partie
    for part in range(1, nb_hx + 3):
        # Récupérer les valeurs de Qdot_tube_back_conv pour la dernière itération pour ce cas et ce maillage
        iteration = limit_big_it - 1
        ratio = ratio_tube_list[mesh][case][iteration][part-1][0]
        tube_back_value = ratio*df_one_mesh_case_list[mesh][case][iteration]['Qdot_tube_back'].loc[f'part{part}']
        tube_back_values.append(tube_back_value)

    # Ajouter les barres pour chaque partie
    fig_comparison.add_trace(go.Bar(
        x=bar_positions + mesh * bar_width - 0.4,
        y=tube_back_values,
        name=f'Qdot_tube_back_conv 1D',
        marker_color=sim_colors[mesh],
        width=bar_width,
        opacity=0.8,
        hovertemplate= f'Simu M{mesh+1} <br>Part{part} <br>'+'%{y:.2f}'
    ))

# Mise en forme du titre et des axes
fig_comparison.update_layout(
    title=f'Qdot_tube_back_conv - Cas {case + 1} - 1D',
    xaxis_title='Parties',
    yaxis_title='Qdot_tube_back_conv (W)',
    barmode='group',  # Regrouper les barres
    legend_title='Simulation'
)

# Affichage de la figure
fig_comparison.show()



## Comparaison des maillages pour Qdot_tube_back_rad // 1D
# Via ratio
# Créer une figure pour comparer les simulations
fig_comparison = go.Figure()

# Pour chaque simulation (chaque maillage)
for mesh in range(nb_mesh):
    # Liste pour stocker les valeurs de Qdot_tube_back_rad pour chaque partie
    tube_back_values = []

    # Pour chaque partie
    for part in range(1, nb_hx + 3):
        # Récupérer les valeurs de Qdot_tube_back_rad pour la dernière itération pour ce cas et ce maillage
        iteration = limit_big_it - 1
        ratio = ratio_tube_list[mesh][case][iteration][part-1][1]
        tube_back_value = ratio*df_one_mesh_case_list[mesh][case][iteration]['Qdot_tube_back'].loc[f'part{part}']
        tube_back_values.append(tube_back_value)

    # Ajouter les barres pour chaque partie
    fig_comparison.add_trace(go.Bar(
        x=bar_positions + mesh * bar_width - 0.4,
        y=tube_back_values,
        name=f'Qdot_tube_back_rad 1D',
        marker_color=sim_colors[mesh],
        width=bar_width,
        opacity=0.8,
        hovertemplate= f'Simu M{mesh+1} <br>Part{part} <br>'+'%{y:.2f}'
    ))

# Mise en forme du titre et des axes
fig_comparison.update_layout(
    title=f'Qdot_tube_back_rad - Cas {case + 1} - 1D',
    xaxis_title='Parties',
    yaxis_title='Qdot_tube_back_rad [W]',
    barmode='group',  # Regrouper les barres
    legend_title='Simulation'
)

# Affichage de la figure
fig_comparison.show()



## Comparaison des maillages pour Qdot_PV_sky // 1D
# Créer une figure pour comparer les simulations
fig_comparison = go.Figure()

# Pour chaque simulation (chaque maillage)
for mesh in range(nb_mesh):
    # Liste pour stocker les valeurs de Qdot_PV_sky pour chaque partie
    PV_values = []

    # Pour chaque partie
    for part in range(1, nb_hx + 3):
        # Récupérer les valeurs de Qdot_PV_sky pour la dernière itération pour ce cas et ce maillage
        iteration = limit_big_it - 1
        PV_value = df_one_mesh_case_list[mesh][case][iteration]['Qdot_PV_sky'].loc[f'part{part}']
        PV_values.append(PV_value)

    # Ajouter les barres pour chaque partie
    fig_comparison.add_trace(go.Bar(
        x=bar_positions + mesh * bar_width - 0.4,
        y=PV_values,
        name=f'Qdot_PV_sky 1D',
        marker_color=sim_colors[mesh],
        width=bar_width,
        opacity=0.8,
        hovertemplate= f'Simu M{mesh+1} <br>Part{part} <br>'+'%{y:.2f}'
    ))

# Mise en forme du titre et des axes
fig_comparison.update_layout(
    title=f'Qdot_PV_sky - Cas {case + 1} - 1D',
    xaxis_title='Parties',
    yaxis_title='Qdot_PV_sky [W]',
    barmode='group',  # Regrouper les barres
    legend_title='Simulation'
)

# Affichage de la figure
fig_comparison.show()